# **Proyecto Final**: *Regresión para predicción de las notas de un curso de portugués*

## Nombre: **Erick Picén**     Carnet: **21000675**

### **Link Video**: https://youtu.be/U8NFOk2dsjc

#### **Justificación**  

*   La realización de los modelos se hace para ejercitar el uso de redes neuronales para regresión y en combinación con PCA.
*   Obtención de mayor experiencia en análisis y predicción de datos que puede ser aplicado para un posible emprendimiento futuro.
*   La realización de este modelo me genera valor a mí como estudiante de Business Ingelligence porque me permite aplicar los conocimientos vistos en clase .

### Preparación del Ambiente

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

### Carga de la Data

In [ ]:
data_path1 = '/content/drive/MyDrive/DataTratadaGradesF.csv'
data = pd.read_csv(data_path1)
data = data.drop(['Unnamed: 0'], axis=1)
#https://www.kaggle.com/datasets/uciml/student-alcohol-consumption?resource=download

In [ ]:
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0.730944,0.833377,1.031695,0.660182,0.648175,-2.666927,1.310216,1.540715,-0.470921,-1.795986,...,0.072606,-0.171647,0.693785,-0.543555,-0.997695,-0.371042,0.106883,-2.459457,-0.226230,11
1,0.730944,0.833377,0.210137,0.660182,0.648175,0.374963,-1.336039,-1.188832,-0.470921,0.816589,...,1.119748,-0.171647,-0.157380,-0.543555,-0.997695,-0.371042,-0.369049,-0.888936,-0.226230,11
2,0.730944,0.833377,-1.432980,0.660182,-1.542792,0.374963,-1.336039,-1.188832,-0.470921,0.816589,...,0.072606,-0.171647,-1.008546,0.538553,0.560678,-0.371042,0.582814,0.215824,0.503327,12
3,0.730944,0.833377,-1.432980,0.660182,0.648175,0.374963,1.310216,-0.278983,-1.619103,-0.651505,...,-0.974536,-1.123771,-1.008546,-0.543555,-0.997695,1.012903,-0.844980,0.956432,0.868105,14
4,0.730944,0.833377,-0.611422,0.660182,0.648175,0.374963,0.428131,0.630866,1.152371,0.816589,...,0.072606,-0.171647,-1.008546,-0.543555,-0.218508,1.012903,-0.844980,-0.153314,0.503327,13


In [ ]:
data.shape

(649, 33)

### Separación Data para Entrenamiento y para Validación

In [ ]:
X = data.iloc[:, data.columns != 'G3']
y = data.loc[:, 'G3']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1234)

### **Función para Generar Arquitectura Dinámica**

In [ ]:
def create_dynamicNeuralNet(layers, activation_fn, tipo, lr, n_inputs, n_clases):
  modelo = Sequential()

  for i, nodes in enumerate(layers):
    if(i == 0):
      modelo.add(Dense(units=nodes, input_dim=n_inputs))
      modelo.add(Activation(activation_fn))
    else:
      modelo.add(Dense(units=nodes))
      modelo.add(Activation(activation_fn))

    if(tipo == "regresion"):
      modelo.compile(loss="mean_squared_error", optimizer=Adam(lr), metrics=['mean_squared_error'])
      out_activation='relu'
      classes_out = 1
    else:
      modelo.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=['accuracy'])
      out_activation='sigmoid'
      classes_out = n_clases[0]
      if(classes_out == 2):
        classes_out = 1
    modelo.add(Dense(units=classes_out, activation=out_activation))
    return modelo


In [ ]:
modeloFinal = KerasRegressor(build_fn=create_dynamicNeuralNet, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


### **Arquitectura 1**

In [ ]:
parametros = {
    'batch_size': [35],
    'activation_fn': ['relu'],
    'epochs': [800],
    'lr': [0.001],
    'layers': [(2, 3, 2)],
    'tipo': ['regresion'],
    'n_inputs': [X_train.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out = GridSearchCV(estimator=modeloFinal, param_grid=parametros)

In [ ]:
grid_result = grid_out.fit(X_train, y_train)

In [ ]:
y_preds_1 = grid_result.predict(X_test)

### Métricas de la Arquitectura 1

In [ ]:
r2r1 =r2_score(y_test, y_preds_1)
r2r1

0.7374939200041192

In [ ]:
MSE = mean_squared_error(y_test, y_preds_1)
RMSE1 = math.sqrt(MSE)
RMSE1 

1.7667921632003816

### **Arquitectura 2**

In [ ]:
parametros2 = {
    'batch_size': [25],
    'activation_fn': ['relu'],
    'epochs': [1000],
    'lr': [0.001],
    'layers': [(5, 5, 3)],
    'tipo': ['regresion'],
    'n_inputs': [X_train.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out2 = GridSearchCV(estimator=modeloFinal, param_grid=parametros2)

In [ ]:
grid_result2 = grid_out2.fit(X_train, y_train)

In [ ]:
y_preds_2 = grid_result2.predict(X_test)

### Métricas Arquitectura 2

In [ ]:
r2r2 =r2_score(y_test, y_preds_2)
r2r2

0.6340758215196312

In [ ]:
MSE = mean_squared_error(y_test, y_preds_2)
RMSE2 = math.sqrt(MSE)
RMSE2 

2.0859857347190585

### **Arquitectura 3**

In [ ]:
parametros3 = {
    'batch_size': [15],
    'activation_fn': ['relu'],
    'epochs': [100],
    'lr': [0.01],
    'layers': [(2, 2, 2)],
    'tipo': ['regresion'],
    'n_inputs': [X_train.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out3 = GridSearchCV(estimator=modeloFinal, param_grid=parametros3)

In [ ]:
grid_result3 = grid_out3.fit(X_train, y_train)

In [ ]:
y_preds_3 = grid_result3.predict(X_test)

### Métricas Arquitectura 3

In [ ]:
r2r3 =r2_score(y_test, y_preds_3)
r2r3

0.6729759034985793

In [ ]:
MSE = mean_squared_error(y_test, y_preds_3)
RMSE3 = math.sqrt(MSE)
RMSE3

1.9719943128167856

### **Arquitectura 4**

In [ ]:
parametros4 = {
    'batch_size': [50],
    'activation_fn': ['relu'],
    'epochs': [480],
    'lr': [0.01],
    'layers': [(3,  20, 15)],
    'tipo': ['regresion'],
    'n_inputs': [X_train.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out4 = GridSearchCV(estimator=modeloFinal, param_grid=parametros4)

In [ ]:
grid_result4 = grid_out4.fit(X_train, y_train)

In [ ]:
y_preds_4 = grid_result4.predict(X_test)

### Métricas Arquitectura 4

In [ ]:
r2r4 =r2_score(y_test, y_preds_4)
r2r4

0.6992773395498678

In [ ]:
MSE = mean_squared_error(y_test, y_preds_4)
RMSE4 = math.sqrt(MSE)
RMSE4

1.8910319141352765

### **Arquitectura 5**

In [ ]:
parametros5 = {
    'batch_size': [75],
    'activation_fn': ['relu'],
    'epochs': [880],
    'lr': [0.001],
    'layers': [(20, 30, 25)],
    'tipo': ['regresion'],
    'n_inputs': [X_train.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out5 = GridSearchCV(estimator=modeloFinal, param_grid=parametros5)

In [ ]:
grid_result5 = grid_out5.fit(X_train, y_train)

In [ ]:
y_preds_5 = grid_result5.predict(X_test)

### Métricas Arquitectura 5

In [ ]:
r2r5 = r2_score(y_test, y_preds_5)
r2r5

0.5548735145662009

In [ ]:
MSE = mean_squared_error(y_test, y_preds_5)
RMSE5 = math.sqrt(MSE)
RMSE5

2.30068678990635

### **Arquitectura 6**

In [ ]:
parametros6 = {
    'batch_size': [10],
    'activation_fn': ['relu'],
    'epochs': [150],
    'lr': [0.01],
    'layers': [(2, 5, 3)],
    'tipo': ['regresion'],
    'n_inputs': [X_train.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out6 = GridSearchCV(estimator=modeloFinal, param_grid=parametros6)

In [ ]:
grid_result6 = grid_out6.fit(X_train, y_train)

In [ ]:
y_preds_6 = grid_result6.predict(X_test)

### Métricas Arquitectura 6

In [ ]:
r2r6 = r2_score(y_test, y_preds_6)
r2r6

0.6907196532445735

In [ ]:
MSE = mean_squared_error(y_test, y_preds_6)
RMSE6 = math.sqrt(MSE)
RMSE6

1.917749784774699

### Resumen

La arquitectura que mejor se desempeño tanto para ajustarse al modelo como para hacer predicciones fue el modelo 6 pero el modelo 3 es técnicamente igual de bueno para ajustarse como para hacer predicciones. La tercera mejor opción es el modelo 4 con 5 puntos menos en el ajuste. Estos 3 modelos se utilizarán en conjunto con PCA

In [ ]:
DFsummary = { 'Arquitectura' : ["Arq1","Arq2","Arq3","Arq4","Arq5","Arq6"],
             'R2': [r2r1,r2r2,r2r3,r2r4,r2r5,r2r6],
             'RMSE': [RMSE1,RMSE2,RMSE3,RMSE4,RMSE5,RMSE6]}
DFsummary = pd.DataFrame(DFsummary, columns =['Arquitectura', 'R2', 'RMSE'])
DFsummary

,Arquitectura,R2,RMSE
0,Arq1,0.737494,1.766792
1,Arq2,0.634076,2.085986
2,Arq3,0.672976,1.971994
3,Arq4,0.699277,1.891032
4,Arq5,0.554874,2.300687
5,Arq6,0.690720,1.917750


### **PCA Principal Component Analisys**

In [ ]:
pca = PCA(n_components=11)

In [ ]:
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
X_train_pca.shape

(519, 11)

In [ ]:
pca.explained_variance_ratio_

array([0.11670044, 0.07628211, 0.05157677, 0.05114657, 0.04530119,
       0.04392138, 0.04275089, 0.03793792, 0.03559743, 0.03534855,
       0.03242967])

In [ ]:
pca.explained_variance_ratio_.cumsum()

array([0.11670044, 0.19298255, 0.24455931, 0.29570588, 0.34100707,
       0.38492845, 0.42767935, 0.46561726, 0.50121469, 0.53656324,
       0.5689929 ])

In [ ]:
np.sum(pca.explained_variance_ratio_)

0.5689929030291697

In [ ]:
np.round(pca.components_, 2)

array([[-0.24, -0.06,  0.16, -0.18, -0.01,  0.01, -0.31, -0.28,  0.18,
         0.12,  0.08, -0.06,  0.21, -0.2 ,  0.23,  0.05, -0.09, -0.01,
         0.09, -0.08, -0.26, -0.15, -0.09, -0.06,  0.08,  0.09,  0.18,
         0.15,  0.01,  0.1 , -0.4 , -0.38],
       [ 0.16, -0.34,  0.09,  0.17, -0.06,  0.02,  0.29,  0.28, -0.2 ,
        -0.1 , -0.03,  0.01, -0.12, -0.16,  0.09,  0.13, -0.  , -0.05,
        -0.12,  0.01, -0.08,  0.23,  0.01,  0.02,  0.19,  0.29,  0.33,
         0.41,  0.13,  0.18, -0.09, -0.09],
       [ 0.11, -0.02,  0.24,  0.12, -0.09, -0.01, -0.2 , -0.26,  0.3 ,
         0.32, -0.13, -0.03,  0.01,  0.03, -0.14,  0.27, -0.28,  0.17,
        -0.03, -0.11,  0.01,  0.11, -0.08,  0.03,  0.02,  0.18,  0.14,
         0.19, -0.13,  0.06,  0.36,  0.35],
       [ 0.24,  0.18,  0.18,  0.31,  0.01, -0.34,  0.05,  0.03,  0.12,
         0.01, -0.1 , -0.07, -0.25,  0.03,  0.11, -0.04,  0.1 ,  0.01,
         0.25, -0.11, -0.07,  0.12, -0.15, -0.27, -0.3 , -0.17, -0.03,
        -0.09, -

### Arquitectura 3 con PCA

In [ ]:
parametros3 = {
    'batch_size': [15],
    'activation_fn': ['relu'],
    'epochs': [100],
    'lr': [0.01],
    'layers': [(2, 2, 2)],
    'tipo': ['regresion'],
    'n_inputs': [X_train_pca.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out3 = GridSearchCV(estimator=modeloFinal, param_grid=parametros3)

In [ ]:
grid_result3 = grid_out3.fit(X_train_pca, y_train)

In [ ]:
y_preds_3pca = grid_result3.predict(X_test_pca)

#### Métricas de la Arquitectura 3 con PCA

In [ ]:
r2r3 =r2_score(y_test, y_preds_3pca)
r2r3

0.7038224884667549

In [ ]:
MSE = mean_squared_error(y_test, y_preds_3pca)
RMSE3 = math.sqrt(MSE)
RMSE3

1.87668689289149

### Arquitectura 4 con PCA

In [ ]:
parametros4 = {
    'batch_size': [50],
    'activation_fn': ['relu'],
    'epochs': [480],
    'lr': [0.01],
    'layers': [(3,  20, 15)],
    'tipo': ['regresion'],
    'n_inputs': [X_train_pca.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out4 = GridSearchCV(estimator=modeloFinal, param_grid=parametros4)

In [ ]:
grid_result4 = grid_out4.fit(X_train_pca, y_train)

In [ ]:
y_preds_4pca = grid_result4.predict(X_test_pca)

### Métricas Arquitectura 4 con PCA

In [ ]:
r2r4 =r2_score(y_test, y_preds_4pca)
r2r4

0.6652710539551608

In [ ]:
MSE = mean_squared_error(y_test, y_preds_4pca)
RMSE4 = math.sqrt(MSE)
RMSE4

1.9950896513046483

### **Arquitectura 6 con PCA**

In [ ]:
parametros6 = {
    'batch_size': [10],
    'activation_fn': ['relu'],
    'epochs': [150],
    'lr': [0.01],
    'layers': [(2, 5, 3)],
    'tipo': ['regresion'],
    'n_inputs': [X_train_pca.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [ ]:
grid_out6 = GridSearchCV(estimator=modeloFinal, param_grid=parametros6)

In [ ]:
grid_result6 = grid_out6.fit(X_train_pca, y_train)

In [ ]:
y_preds_6pca = grid_result6.predict(X_test_pca)

#### Métricas Arquitectura 6

In [ ]:
r2r6 = r2_score(y_test, y_preds_6pca)
r2r6

0.7019774077049136

In [ ]:
MSE = mean_squared_error(y_test, y_preds_6pca)
RMSE6 = math.sqrt(MSE)
RMSE6

1.8825233638033994

In [ ]:
DFsummaryF = { 'Arquitectura con PCA' : ["Arq3","Arq4","Arq6"],
             'R2': [r2r3,r2r4,r2r6],
             'RMSE': [RMSE3,RMSE4,RMSE6]}
DFsummaryF = pd.DataFrame(DFsummaryF, columns =['Arquitectura con PCA', 'R2', 'RMSE'])
DFsummaryF

,Arquitectura con PCA,R2,RMSE
0,Arq3,0.703822,1.876687
1,Arq4,0.665271,1.995090
2,Arq6,0.701977,1.882523


### **CONCLUSIÓN**

*   Luego de aplicar PCA a las 3 mejores arquitecturas se determinó que 11 componentes es la cantidad que mejor balance ofrece entre métricas y cantidad de variables, con poco menos de 1/3 del total de variabes se obtuvieron casi las mismas métricas que con las 33 columnas originales que contiene el dataset. 



### **RECOMENDACIONES**



*   Sería posible intentar otras arquitecturas con más capas, un learning rate mayor.
*   Dado que las notas se encuentran en el rango de 0 a 20 es posible crear un modelo de clasificación.
*   Debido a que dataset tiene una gran cantidad de variables es posible aplicarle aprendizaje no supervisado.
*   Cuando se incrementó demasiado el número de epocas, la cantidad de capas y el learning rate, se obtuvieron valores de R2 más pobres
*   Al utilizar un múmero mayor de 15 componentes con PCA, las redes disminuyeron su desempeño obteniendo R2 negativos.



